# Création et résolution de labyrinthes

## 1°) Introduction

In [1]:
import random

########
class UneCellule:
    """
    définition d'une cellule
    """
    
    def __init__(self,x, y):
        """
        créer une cellule positionnée en (x=ligne, y=colonne)
        """
        
        self.x = x
        self.y = y
        #les murs sont dans l'ordre : N, S, E, O. 
        #la valeur est à True si un mur est présent, False sinon
        self.murs = {'N': True, 'S': True, 'E': True, 'O': True}

########       
class Grille :
    """
    construction d'une grille de cellules
    """
    
    def __init__(self, nl, nc):
        """
        construction d'une grille de dimension (nl, nc)
        """
        
        self.nl = nl
        self.nc = nc
        self.cadrillage = []
        for i in range(nl):
            GrilleLigne=[]
            for j in range(nc):
                GrilleLigne.append(UneCellule(i,j))
            self.cadrillage.append(GrilleLigne)
        
        
    def cellule(self, x, y):
        """
        retourne la cellule de la grille de position (x=ligne, y=colonne)
        """
        
        return self.cadrillage[x][y]
    
    def __str__(self):
        """
        retourne une chaine de caractères représentant le labyrinthe
        pour les cellules touchant le bord gauche ou le bord du haut de la grille, les 4 murs sont représentés.
        pour les autres, seuls les murs Est et Sud sont représentés
        """
        
        laby_lignes = []
        laby_l = ['+']
        for y in range(self.nc):
            if self.cadrillage[0][y].murs['N']:
                laby_l.append('---+')
            else :
                laby_l.append('   +')
        laby_lignes.append(''.join(laby_l))
                              
              
        for x in range(0,self.nl):
            if self.cadrillage[x][0].murs['O'] :
                laby_l = ['|']
            else :
                laby_l = [' ']
            for y in range(self.nc):
                if self.cadrillage[x][y].murs['E']:
                    laby_l.append('   |')
                else:
                    laby_l.append('    ')
            laby_lignes.append(''.join(laby_l))
            laby_l = ['+']
            for y in range(self.nc):
                if self.cadrillage[x][y].murs['S']:
                    laby_l.append('---+')
                else:
                    laby_l.append('   +')
            laby_lignes.append(''.join(laby_l))
        
        #laby_lignes.append('\n')
        return '\n'.join(laby_lignes)

## 1_4 Premier exercice 

In [2]:
# Permet de convertir une orientation en son opposé
def dirInvert(orientation: str)->str:
    r=""
    if(orientation == 'N'):
        r = "S"
    if(orientation == 'S'):
        r = "N"
    if(orientation == 'O'):
        r = "E"
    if(orientation == 'E'):
        r = "O"
    return r

#Permet de convertir une orientation pour toujours à droite
def dirDroite(orientation: str) -> tuple:
    r = ""
    if (orientation == 'N'):
        r = "E"
    elif (orientation == 'S'):
        r = "O"
    elif (orientation == 'O'):
        r = "N"
    elif (orientation == 'E'):
        r = "S"
    return r

# Permet de convertir une orientation en ses coordonnées
def dirConvert(orientation : str)-> tuple:
    r=""
    if(orientation == 'N'):
        r = (-1,0)
    if(orientation == 'S'):
        r = (1,0)
    if(orientation == 'O'):
        r = (0,-1)
    if(orientation == 'E'):
        r = (0,1)
    return r

def convertCoordToOrientation(coord):
    res = ""
    if coord == (0, -1):
        res = "O"
    elif coord == (0, 1):
        res = "E"
    elif coord == (-1, 0):
        res = "N"
    elif coord == (1, 0):
        res = "S"
    return res

# Permet d'additionner 2 tuples entre eux
def addTuple(coord:tuple,dir:tuple)->tuple:
    return (coord[0] + dir[0], coord[1] + dir[1])

# Permet de soustraire 2 tuples entre eux
def subTuple(coord:tuple,dir:tuple)->tuple:
    return (coord[0] - dir[0], coord[1] - dir[1])

# Permet de savoir si une coordonnée est bien dans le labyrinthe
def estDansLaby(grille: Grille, coord) -> bool:
    return not (coord[0] < 0 or coord[0] >= grille.nl or coord[1] < 0 or coord[1] >= grille.nc)

# Permet de retourner un tableau 2D remplit avec False et sa coordonnée
def create2d(grille: Grille) -> list:
    res = []
    for i in range(grille.nl):
        line = []
        for j in range(grille.nc):
            line.append([False, (i, j)])
        res.append(line)
    return res

# Permet de retourner la liste des voisins d'une cellule qui n'a pas été traité
def getVoisin(table: list, grille: Grille, coord: tuple) -> list:
    res = []
    if (estDansLaby(grille, coord)):
        voisins = [(coord[0]-1, coord[1]), (coord[0]+1, coord[1]), (coord[0], coord[1]+1), (coord[0], coord[1]-1)]
        for i in range(0, len(voisins)):
            if estDansLaby(grille, voisins[i]):
                if (table[voisins[i][0]][voisins[i][1]][0] == False):
                    res.append(voisins[i])
    return res

# Permet de retourner la liste des murs qui sont à False donc qui ne sont pas présent
def getOrientationPasMur(grille: Grille, coord: tuple) -> list:
    res = []
    for i in grille.cellule(coord[0], coord[1]).murs:
        if (grille.cellule(coord[0], coord[1]).murs[i] == False):
            res.append(i)
    return res

In [3]:
def effaceMur(grille, coord , orientation):
    # On convertit l'orientation en coordonnée
    ori = dirConvert(orientation)
    
    # On cherche l'inverse de l'orientation donnée
    ori1= dirInvert(orientation)
    
    # On calcule la case qui verra son mur supprimé
    case1 = addTuple(coord, ori)
    
    # Si la case que l'on veut supprimer existe dans le labyrinthe
    if estDansLaby(grille, coord):
        # On supprime le mur voulu
        grille.cellule(coord[0],coord[1]).murs[orientation]=False
        
        # Si la case voisine que l'on veut supprimer existe dans le labyrinthe
        if estDansLaby(grille, case1):
            # On supprime le mur voulu
            grille.cellule(case1[0],case1[1]).murs[ori1]=False
    return grille

## 2°) Construction de labyrinthes parfaits 

### 2_1 Algorithme de l'arbre binaire 

In [4]:
def arbreBinaire(grille: Grille) -> Grille:
    # On fait une boucle sur les lignes
    for x in range(0, grille.nl):
        # On fait une boucle sur les colonnes
        for y in range(0, grille.nc):
            # On assigne les coordonnées à une variable
            coord = (x, y)
            # Si coord n'est pas égal à la dernière case en bas à droite du labyrinthe
            if not (coord[0] == grille.nl-1 and coord[1] == grille.nc-1):
                # Si y est égal au nombre de colonne - 1 de la grille
                if (coord[1] == grille.nc-1):
                    # On supprime le mur S de cette coordonnée
                    effaceMur(grille,coord,'S')
                # Sinon si x est égal au nombre de ligne - 1 de la grille
                elif (coord[0] == grille.nl-1):
                    # On supprime le mur E de cette coordonnée
                    effaceMur(grille,coord,'E')
                # Sinon on suprimme un mur de manière aléatoire entre E et S
                else:
                    effaceMur(grille,coord,random.choice(['E', 'S']))
    return grille

In [5]:
ab = Grille(4,6)
print(arbreBinaire(ab))

+---+---+---+---+---+---+
|   |   |   |       |   |
+   +   +   +---+   +   +
|   |           |       |
+   +---+---+   +---+   +
|           |   |   |   |
+---+---+   +   +   +   +
|                       |
+---+---+---+---+---+---+


### 2_2 Algorithme sidewinder 

In [6]:
def sidewinder(grille: Grille) -> Grille:
    # On fait une boucle sur les lignes
    for i in range(grille.nl):
        # On fait une boucle sur les colonnes
        for j in range(grille.nc):
            # On regarde si i et j ne sont pas égal respectivement à la dernière ligne et à la dernière colonne
            if (i != grille.nl - 1 or j != grille.nc - 1):
                # Si la ligne est bien la dernière alors
                if (i == grille.nl - 1):
                    # On supprime le mur E aux coordonnées (i, j)
                    effaceMur(grille, (i, j), 'E')
                # Sinon si la colonne est bien la dernière alors
                elif ( j == grille.nc - 1 ):
                    # On supprime le mur S aux coordonnées (i, j)
                    effaceMur(grille,(i,j),'S')
                # Sinon
                else:
                    # On choisit aléatoirement 0 ou 1
                    rd = random.choice([0, 1])
                    # Si c'est 0 alors
                    if (rd == 0):
                        # On supprime le mur E aux coordonnées (i, j)
                        effaceMur(grille, (i, j), 'E')
                    # Sinon
                    else:
                        # On regarde toute les cellules qui ont formé un passage, càd sans mur entre 2 cellules
                        adjacent = [(i, j)]
                        m = j-1
                        while (grille.cellule(i, m).murs['E'] == False):
                            adjacent.append((i, m))
                            m -= 1
                        # On choisit aléatoire parmi le passage, une cellule et on détruit son mur S
                        effaceMur(grille, random.choice(adjacent), 'S')
    return grille 

In [7]:
SW = Grille(4,6)
print(sidewinder(SW))

+---+---+---+---+---+---+
|   |       |   |       |
+   +   +---+   +---+   +
|           |   |       |
+   +---+---+   +---+   +
|   |   |   |   |       |
+   +   +   +   +---+   +
|                       |
+---+---+---+---+---+---+


### 2_3 Algorithme d'exploration exhaustive 

In [8]:
def explorationE(grille: Grille) -> Grille:
    
    # On crée un tableau 2D avec un booléen de visite et leurs coordonnées
    visite = create2d(grille)
    
    # On initialise la liste pile
    pile = []
    
    # On choisit une cellule aléatoire parmi le tableau 2D
    cell = random.choice(random.choice(visite))[1]
    
    # On ajoute à la pile cette première cellule
    pile.append(cell)
    
    # On mets dans le tableau 2D que cette cellule a été visité
    visite[cell[0]][cell[1]][0] = True
    
    # Tant que la pile n'est pas vide
    while (len(pile) != 0):
        # On récupère la liste des voisins du premier élément de la pile
        voisins = getVoisin(visite, grille, pile[0])
        
        # Si la longueur de cette liste n'est pas égal à 0
        if (len(voisins) != 0):
            
            # On choisit une cellule aléatoirement parmi cette liste
            voisin = random.choice(voisins)
            
            # On supprime le mur qui va de ce voisin vers la cellule étudiée dans la pile
            effaceMur(grille, pile[0], convertCoordToOrientation(subTuple(voisin, pile[0])))
            
            # On mets dans le tableau 2D que cette cellule a été visité
            visite[voisin[0]][voisin[1]][0] = True
            
            # On insert au début de pile cette cellule voisine choisit aléatoirement
            pile.insert(0, voisin)
        
        # Sinon
        else:
            # On supprime le premier élément de la pile car on a tout étudié
            del pile[0]
    return grille

In [9]:
grilleTest = Grille(4,6)
exp = explorationE(grilleTest)
print(exp)

+---+---+---+---+---+---+
|               |       |
+---+---+   +   +---+   +
|       |   |           |
+   +---+   +---+---+   +
|       |   |           |
+---+   +   +   +---+---+
|           |           |
+---+---+---+---+---+---+


### 2_4 Bonus 

In [10]:
def fusion(grille):
    idLst = [[i+1+j*grille.nc for i in range(grille.nc)] for j in range(grille.nl)]
    step = 0
    maxStep = grille.nc*grille.nl-1
    while step < maxStep:
        orientation = random.choice(['N', 'S', 'E', 'O'])
        cell = (random.randint(0, grille.nl-1), random.randint(0, grille.nc-1))
        idCell = idLst[cell[0]][cell[1]]
        step += 1

In [11]:
print(fusion(grilleTest))

None


## 3°) Résolution d'un labyrinthe parfait 

### 3_1 Introduction 

In [12]:
def attenantes(grille: Grille,coord: tuple) -> list:
    # On initialise la liste de résultat
    res = []
    # Si la coordonnée est bien dans le labyrinthe
    if (estDansLaby(grille, coord)):
        # On récupère la liste des orientations où il n'y a pas de mur
        orientationPasMur = getOrientationPasMur(grille,coord)
        # Pour chaque élément dans cette liste
        for orientation in orientationPasMur:
            # On récupère le voisin en fonction de sa coordonnée
            coordVoisin = addTuple(coord, dirConvert(orientation))
            # Et si ce voisin est bien dans le labyrinthe
            if estDansLaby(grille, coordVoisin):
                # Alors on l'ajoute à la liste de résultat
                res.append(coordVoisin)
    return res

In [13]:
laby = explorationE(Grille(5,7))
effaceMur(laby,(0,0),'O')
effaceMur(laby,(4,6),'E')
print(laby)
print(attenantes(laby,(1,3)))
print(attenantes(laby,(1,1)))

+---+---+---+---+---+---+---+
            |               |
+   +---+   +---+---+---+   +
|       |   |           |   |
+   +   +   +   +---+   +   +
|   |   |   |       |       |
+---+   +   +---+   +---+---+
|       |       |           |
+   +---+---+   +---+---+   +
|   |                        
+---+---+---+---+---+---+---+
[(2, 3), (1, 4)]
[(2, 1), (1, 0)]


### 3_2 Calcul des distances depuis (0,0) 

In [14]:
def distance(grille: Grille) -> list:
    # On crée un tableau 2D remplit de 0
    D = [[ 0 for i in range(grille.nc)] for j in range (grille.nl)]
    # On crée un tableau 2D remplit de False, elle va nous servir à savoir si on a visité une cellule
    visite = [[ False for i in range(grille.nc)] for j in range (grille.nl)]
    # On initialise une liste file avec la première case à l'intérieur
    file = [(0, 0)]
    # On mets dans la matrice pour la première case, 0
    D[0][0] = 0
    # Tant que la liste file n'est pas vide
    while (len(file) != 0):
        # L'élément traité sera le premier élément de la file
        element = file[0]
        # On initialise une liste pour stocker les cellules attenantes qui n'ont pas été visité
        cellAttenantes = []
        # Pour chaque cellules attenantes
        for coord in attenantes(grille, element):
            # On vérifie si elle n'a pas été visité
            if visite[coord[0]][coord[1]] == False:
                # Si c'est le cas, on ajoute les coordonnées à la liste
                cellAttenantes.append(coord)
                # On mets à la fin de la liste file, la coordonnée attenante
                file.append(coord)
                # On ajoute le premier élément de la pile + 1 à cette cellule dans la matrice
                D[coord[0]][coord[1]] = D[element[0]][element[1]] + 1
        # On dit qu'on a visité la première cellule de file
        visite[file[0][0]][file[0][1]] = True
        # On supprime ensuite cette même cellule du file
        del file[0]
    return D

In [15]:
print(laby)
print(distance(laby))

+---+---+---+---+---+---+---+
            |               |
+   +---+   +---+---+---+   +
|       |   |           |   |
+   +   +   +   +---+   +   +
|   |   |   |       |       |
+---+   +   +---+   +---+---+
|       |       |           |
+   +---+---+   +---+---+   +
|   |                        
+---+---+---+---+---+---+---+
[[0, 1, 2, 25, 24, 23, 22], [1, 2, 3, 16, 17, 18, 21], [2, 3, 4, 15, 14, 19, 20], [5, 4, 5, 6, 13, 12, 11], [6, 9, 8, 7, 8, 9, 10]]


### 3_3 Résolution 

In [16]:
def resolution(grille: Grille) -> list:
    # On récupère la liste des distances de chaques cellules
    grilleDist = distance(grille)
    # On récupère la dernière case
    current = (len(grilleDist)-1, len(grilleDist[len(grilleDist)-1])-1)
    # On l'ajoute à la liste de résultat
    res = [current]
    # Tant que la distance d'une cellule est supérieur à 0
    while (grilleDist[current[0]][current[1]] > 0):
        # On récupère la liste des cellules attenantes à la case traité
        attLst = attenantes(grille, current)
        # Pour chaque cellule attenante
        for co in attLst:
            # On récupère la distance de la cellule attenante
            distanceAtte = grilleDist[co[0]][co[1]]
            # Si cette distance est égal à la distance de la cellule courante - 1
            if (distanceAtte == grilleDist[current[0]][current[1]] - 1):
                # Alors la cellule utilisée va devenir cette cellule attenante
                current = co
                # Puis on l'ajoute à la liste de résultat
                res.append(co)
    # On inverse la liste de résultat
    res.reverse()
    return res

In [17]:
print(laby,'\n')
print(resolution(laby))

+---+---+---+---+---+---+---+
            |               |
+   +---+   +---+---+---+   +
|       |   |           |   |
+   +   +   +   +---+   +   +
|   |   |   |       |       |
+---+   +   +---+   +---+---+
|       |       |           |
+   +---+---+   +---+---+   +
|   |                        
+---+---+---+---+---+---+---+ 

[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2), (3, 3), (4, 3), (4, 4), (4, 5), (4, 6)]


### 3_4 BONUS 

In [18]:
def parcoursMD(grille: Grille) -> list:
    # Initialisation de la première cellule
    current = (0, 0)
    # Initialisation de la direction de départ
    orientation = 'E'
    # Initialisation de la liste de résultat
    res = []
    # Tant que la cellule de base n'est pas la dernière
    while (current[0] != grille.nl - 1 or current[1] != grille.nc - 1):
        # On ajoute à la liste de résulat la cellule de base
        res.append(current)
        # On change la direction en fonction de la direction droite
        if not (grille.cellule(current[0], current[1]).murs[dirDroite(orientation)]):
            direction = dirDroite(orientation)
        elif not (grille.cellule(current[0], current[1]).murs[dirInvert(dirDroite(orientation))]):
            direction = dirInvert(dirDroite(orientation))
        elif not (grille.cellule(current[0], current[1]).murs[orientation]):
            pass
        else:
            orientation = dirInvert(orientation)
        # La cellule de base devient le voisin de la cellule précedente en fonction de l'orientation
        current = addTuple(current,convertOrientationToCoord(orientation))
    return res

In [19]:
print(laby,'\n')
print(parcoursMD(laby))

+---+---+---+---+---+---+---+
            |               |
+   +---+   +---+---+---+   +
|       |   |           |   |
+   +   +   +   +---+   +   +
|   |   |   |       |       |
+---+   +   +---+   +---+---+
|       |       |           |
+   +---+---+   +---+---+   +
|   |                        
+---+---+---+---+---+---+---+ 



NameError: name 'convertOrientationToCoord' is not defined